# Saving for Retirement
- Let's assume we want to save $10,000 every year
- Our initial balance is zero
- How much would our account grow over 30 years if we earn a rate of 5%?

In [1]:
import numpy as np
import pandas as pd
!pip install numpy_financial
import numpy_financial as npf
import plotly.graph_objects as go


[notice] A new release of pip available: 22.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
RATE     = 0.05
N_SAVING = 30
PMT      = 10000.0
PV       = 0.0

acct = pd.DataFrame(dtype=float,columns=['begbal','capgain','deposit','endbal'],index=np.arange(1,N_SAVING+1))
acct.deposit = PMT
for t in acct.index:
    if t==1:
        acct.loc[t,'begbal'] = PV
    else:
        acct.loc[t,'begbal'] = acct.loc[t-1,'endbal']
    acct.loc[t,'capgain'] = acct.loc[t,'begbal']*RATE
    acct.loc[t,'endbal'] = acct.loc[t,'begbal'] + acct.loc[t,'capgain'] + acct.loc[t,'deposit']

pd.options.display.float_format = '${:,.2f}'.format
acct

,begbal,capgain,deposit,endbal
1,$0.00,$0.00,"$10,000.00","$10,000.00"
2,"$10,000.00",$500.00,"$10,000.00","$20,500.00"
3,"$20,500.00","$1,025.00","$10,000.00","$31,525.00"
4,"$31,525.00","$1,576.25","$10,000.00","$43,101.25"
5,"$43,101.25","$2,155.06","$10,000.00","$55,256.31"
6,"$55,256.31","$2,762.82","$10,000.00","$68,019.13"
7,"$68,019.13","$3,400.96","$10,000.00","$81,420.08"
8,"$81,420.08","$4,071.00","$10,000.00","$95,491.09"
9,"$95,491.09","$4,774.55","$10,000.00","$110,265.64"
10,"$110,265.64","$5,513.28","$10,000.00","$125,778.93"


In [3]:
# Of course, we could do this in one step using a financial function
npf.fv(RATE, N_SAVING, -PMT, -PV)

664388.4750301335

In [4]:
# If we wanted to be able to withdraw $W=100,000 each year for 20 years, 
# how much do we need to save for the first 30 years

WITHDRAWAL = 100000.0
N_WITHDRAW = 20


In [5]:
# First, we find how much we'd need to have saved as of t=30:

end_savings_bal = npf.pv(RATE,N_WITHDRAW,-WITHDRAWAL,0)
print(f'{end_savings_bal:,.2f}')

1,246,221.03


In [6]:
# Second, we solve a PMT problem to determine the amount we need to save:

pmt = npf.pmt(RATE,N_SAVING,PV,-end_savings_bal)
print(f'{pmt:,.2f}')

18,757.41


In [7]:
#Let's set up a bank account to make sure we are getting a zero ending balance

acct = pd.DataFrame(dtype=float,columns=['begbal','capgain','deposit','withdraw','endbal'],index=np.arange(1,N_SAVING + N_WITHDRAW + 1))
acct.deposit = np.where(acct.index <= N_SAVING, PMT,0)
acct.withdraw= np.where(acct.index > N_SAVING, -WITHDRAWAL,0)
for t in acct.index:
    if t==1:
        acct.loc[t,'begbal'] = PV
    else:
        acct.loc[t,'begbal'] = acct.loc[t-1,'endbal']
    acct.loc[t,'capgain'] = acct.loc[t,'begbal']*RATE
    acct.loc[t,'endbal']  = acct.loc[t,'begbal'] + acct.loc[t,'capgain'] + acct.loc[t,'deposit'] + acct.loc[t,'withdraw']
acct

,begbal,capgain,deposit,withdraw,endbal
1,$0.00,$0.00,"$10,000.00",$0.00,"$10,000.00"
2,"$10,000.00",$500.00,"$10,000.00",$0.00,"$20,500.00"
3,"$20,500.00","$1,025.00","$10,000.00",$0.00,"$31,525.00"
4,"$31,525.00","$1,576.25","$10,000.00",$0.00,"$43,101.25"
5,"$43,101.25","$2,155.06","$10,000.00",$0.00,"$55,256.31"
6,"$55,256.31","$2,762.82","$10,000.00",$0.00,"$68,019.13"
7,"$68,019.13","$3,400.96","$10,000.00",$0.00,"$81,420.08"
8,"$81,420.08","$4,071.00","$10,000.00",$0.00,"$95,491.09"
9,"$95,491.09","$4,774.55","$10,000.00",$0.00,"$110,265.64"
10,"$110,265.64","$5,513.28","$10,000.00",$0.00,"$125,778.93"
